Import pandas library, aliased as 'pd'

In [2]:
import pandas as pd

In [3]:
houses_columnwise = {
    "price_approx_usd": [114910.23, 423912.12, 23839.23,325912.32,83235.12],
    "surface_covered_in_m2":[128.0,239.2,83.2, 934.2,122.1],
    "rooms": [4.0,3.0,2.0,3.0,3.0]
}

In [4]:
houses_columnwise

{'price_approx_usd': [114910.23, 423912.12, 23839.23, 325912.32, 83235.12],
 'surface_covered_in_m2': [128.0, 239.2, 83.2, 934.2, 122.1],
 'rooms': [4.0, 3.0, 2.0, 3.0, 3.0]}

In [5]:
df_houses = pd.DataFrame(houses_columnwise)
df_houses

,price_approx_usd,surface_covered_in_m2,rooms
0,114910.23,128.0,4.0
1,423912.12,239.2,3.0
2,23839.23,83.2,2.0
3,325912.32,934.2,3.0
4,83235.12,122.1,3.0


In [6]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.utils.validation import check_is_fitted

In [7]:
df = pd.read_csv("mexico/mexico-city-real-estate-1.csv")
df

,Unnamed: 0,operation,property_type,place_with_parent_names,lat-lon,price,currency,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url
0,1,sell,apartment,|Miguel Hidalgo|Distrito Federal|México|,"23.634501,-102.552788",5500000.0,MXN,5450245.50,289775.66,NaN,54.0,NaN,101851.851900,NaN,NaN,NaN,http://miguel-hidalgo-df.properati.com.mx/o3zb...
1,2,sell,house,|Iztapalapa|Distrito Federal|México|,"19.31033,-99.068557",1512000.0,MXN,1498321.97,79661.96,NaN,80.0,NaN,18900.000000,NaN,NaN,NaN,http://iztapalapa.properati.com.mx/q7t0_venta_...
2,3,sell,apartment,|Tlalpan|Distrito Federal|México|,"19.279771,-99.234597",926667.0,MXN,918284.00,48822.82,NaN,100.0,NaN,9266.670000,NaN,NaN,NaN,http://tlalpan.properati.com.mx/qbi4_venta_dep...
3,4,sell,apartment,|Miguel Hidalgo|Distrito Federal|México|,"23.634501,-102.552788",6410000.0,MXN,6352013.39,337720.36,NaN,135.0,NaN,47481.481480,NaN,NaN,NaN,http://miguel-hidalgo-df.properati.com.mx/opeq...
4,5,sell,apartment,|Benito Juárez|Quintana Roo|México|,"21.1902642,-86.8198375",875000.0,USD,16457437.50,875000.00,0.0,263.0,NaN,3326.996198,NaN,NaN,NaN,http://cancun.properati.com.mx/hg4t_venta_depa...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4623,4624,sell,apartment,|Benito Juárez|Quintana Roo|México|,"20.797201,-86.967773",200000.0,USD,3761700.00,200000.00,NaN,240.0,NaN,833.333333,NaN,1.0,NaN,http://cancun.properati.com.mx/fmik_venta_depa...
4624,4625,sell,apartment,|Tlalpan|Distrito Federal|México|,"19.303758,-99.223733",10000000.0,MXN,10144274.75,539345.23,200.0,NaN,2696.726150,NaN,NaN,NaN,NaN,http://lomas-del-pedregal.properati.com.mx/pym...
4625,4626,sell,house,|Solidaridad|Quintana Roo|México|,"20.613375,-87.089897",480000.0,USD,9028080.00,480000.00,451.0,245.0,1064.301552,1959.183673,2.0,NaN,NaN,http://playa-del-carmen.properati.com.mx/bvwt_...
4626,4627,sell,apartment,|Cuauhtémoc|Distrito Federal|México|,"23.634501,-102.552788",2453800.0,MXN,2431602.18,129282.09,NaN,67.0,NaN,36623.880600,NaN,NaN,NaN,http://cuauhtemoc.properati.com.mx/p2op_venta_...


In [8]:
def wrangle(filepath):
    df = pd.read_csv(filepath)
    
    rem_parNames = df['place_with_parent_names'].str.contains('Distrito Federal')
    rem_apt = df['property_type'] == 'apartment'
    rem_price = df['price_aprox_usd'] < 400_000
    rem_currency = df['currency'] == 'MXN'
    df = df[rem_parNames & rem_apt & rem_price & rem_currency]
    df[['lat', 'lon']] = df['lat-lon'].str.split(',', expand= True).astype(float)
    df.drop(columns= 'lat-lon', inplace=True)
    
    low, high = df["surface_covered_in_m2"].quantile([0.1,0.9])
    rem_area = df["surface_covered_in_m2"].between(low,high)
    df = df[rem_area]
    
    return df


In [9]:
frame1 = wrangle('mexico/mexico-city-real-estate-1.csv')
frame1.head()

,Unnamed: 0,operation,property_type,place_with_parent_names,price,currency,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,lat,lon
0,1,sell,apartment,|Miguel Hidalgo|Distrito Federal|México|,5500000.0,MXN,5450245.50,289775.66,NaN,54.0,NaN,101851.85190,NaN,NaN,NaN,http://miguel-hidalgo-df.properati.com.mx/o3zb...,23.634501,-102.552788
2,3,sell,apartment,|Tlalpan|Distrito Federal|México|,926667.0,MXN,918284.00,48822.82,NaN,100.0,NaN,9266.67000,NaN,NaN,NaN,http://tlalpan.properati.com.mx/qbi4_venta_dep...,19.279771,-99.234597
5,6,sell,apartment,|Miguel Hidalgo|Distrito Federal|México|,4416000.0,MXN,4376051.62,232663.51,NaN,87.0,NaN,50758.62069,NaN,NaN,NaN,http://anzures.properati.com.mx/opct_venta_dep...,19.432657,-99.177444
6,7,sell,apartment,|Benito Juárez|Distrito Federal|México|,3150000.0,MXN,3122244.47,166001.78,NaN,100.0,NaN,31500.00000,NaN,NaN,NaN,http://benito-juarez.properati.com.mx/r3a9_ven...,19.367025,-99.170349
11,12,sell,apartment,|Benito Juárez|Distrito Federal|México|,1784569.0,MXN,1768425.20,94022.66,NaN,57.0,NaN,31308.22807,NaN,NaN,NaN,http://benito-juarez.properati.com.mx/pdji_ven...,23.634501,-102.552788


In [10]:
frame1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1900 entries, 0 to 4627
Data columns (total 18 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Unnamed: 0                  1900 non-null   int64  
 1   operation                   1900 non-null   object 
 2   property_type               1900 non-null   object 
 3   place_with_parent_names     1900 non-null   object 
 4   price                       1900 non-null   float64
 5   currency                    1900 non-null   object 
 6   price_aprox_local_currency  1900 non-null   float64
 7   price_aprox_usd             1900 non-null   float64
 8   surface_total_in_m2         283 non-null    float64
 9   surface_covered_in_m2       1900 non-null   float64
 10  price_usd_per_m2            156 non-null    float64
 11  price_per_m2                1900 non-null   float64
 12  floor                       0 non-null      float64
 13  rooms                       8 non-null

In [11]:
frame1.shape

(1900, 18)

In [12]:
len(df.columns)

17

In [13]:
df.columns

Index(['Unnamed: 0', 'operation', 'property_type', 'place_with_parent_names',
       'lat-lon', 'price', 'currency', 'price_aprox_local_currency',
       'price_aprox_usd', 'surface_total_in_m2', 'surface_covered_in_m2',
       'price_usd_per_m2', 'price_per_m2', 'floor', 'rooms', 'expenses',
       'properati_url'],
      dtype='object')

In [14]:
frame1['currency'].value_counts()

currency
MXN    1900
Name: count, dtype: int64

In [15]:
frame1['property_type'].value_counts()

property_type
apartment    1900
Name: count, dtype: int64

In [16]:
frame1['place_with_parent_names'].str.contains("Distrito Federal").value_counts()

place_with_parent_names
True    1900
Name: count, dtype: int64

In [17]:
frame2 = wrangle('mexico/mexico-city-real-estate-2.csv')
frame2

,Unnamed: 0,operation,property_type,place_with_parent_names,price,currency,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,lat,lon
0,1,sell,apartment,|Gustavo A. Madero|Distrito Federal|México|,1200000.0,MXN,1189144.46,63223.78,88.0,88.0,718.452046,13636.363640,NaN,NaN,NaN,http://gustavo-a-madero.properati.com.mx/r644_...,19.516777,-99.160149
2,3,sell,apartment,|Álvaro Obregón|Distrito Federal|México|,4559261.0,MXN,4518016.68,240211.43,NaN,126.0,NaN,36184.611110,NaN,NaN,NaN,http://alvaro-obregon.properati.com.mx/jh8j_ve...,19.332280,-99.243934
4,5,sell,apartment,|Benito Juárez|Distrito Federal|México|,1730000.0,MXN,1714349.82,91147.61,NaN,100.0,NaN,17300.000000,NaN,NaN,NaN,http://benito-juarez.properati.com.mx/q5my_ven...,19.393650,-99.147690
17,18,sell,apartment,|Benito Juárez|Distrito Federal|México|,1694000.0,MXN,1678675.55,89250.90,NaN,90.0,NaN,18822.222220,NaN,NaN,NaN,http://benito-juarez.properati.com.mx/nb7e_ven...,19.383327,-99.152712
19,20,sell,apartment,|Álvaro Obregón|Distrito Federal|México|,757073.0,MXN,750224.23,39887.51,NaN,60.0,NaN,12617.883330,NaN,NaN,NaN,http://alvaro-obregon.properati.com.mx/pd6s_ve...,19.388280,-99.195529
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4612,4613,sell,apartment,|Cuauhtémoc|Distrito Federal|México|,1150000.0,MXN,1139596.67,60589.45,NaN,65.0,NaN,17692.307690,NaN,NaN,NaN,http://cuauhtemoc.properati.com.mx/q59a_venta_...,23.634501,-102.552788
4616,4617,sell,apartment,|Azcapotzalco|Distrito Federal|México|,1785000.0,MXN,1768852.34,94045.37,NaN,65.0,NaN,27461.538460,NaN,NaN,NaN,http://azcapotzalco.properati.com.mx/o2tc_vent...,19.472155,-99.161186
4620,4621,sell,apartment,|Tlalpan|Distrito Federal|México|,1113000.0,MXN,1102931.38,58640.05,NaN,80.0,NaN,13912.500000,NaN,NaN,NaN,http://villa-lazaro-cardenas.properati.com.mx/...,19.295734,-99.146996
4623,4624,sell,apartment,|Cuauhtémoc|Distrito Federal|México|,739000.0,MXN,732314.77,38935.31,NaN,76.0,NaN,9723.684211,NaN,NaN,NaN,http://cuauhtemoc.properati.com.mx/izni_venta_...,23.634501,-102.552788


In [18]:
frame2.shape

(1856, 18)

In [19]:
frame1.shape

(1900, 18)

In [20]:
df = pd.concat([frame1,frame2], ignore_index=True)
df

,Unnamed: 0,operation,property_type,place_with_parent_names,price,currency,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,lat,lon
0,1,sell,apartment,|Miguel Hidalgo|Distrito Federal|México|,5500000.0,MXN,5450245.50,289775.66,NaN,54.0,NaN,101851.851900,NaN,NaN,NaN,http://miguel-hidalgo-df.properati.com.mx/o3zb...,23.634501,-102.552788
1,3,sell,apartment,|Tlalpan|Distrito Federal|México|,926667.0,MXN,918284.00,48822.82,NaN,100.0,NaN,9266.670000,NaN,NaN,NaN,http://tlalpan.properati.com.mx/qbi4_venta_dep...,19.279771,-99.234597
2,6,sell,apartment,|Miguel Hidalgo|Distrito Federal|México|,4416000.0,MXN,4376051.62,232663.51,NaN,87.0,NaN,50758.620690,NaN,NaN,NaN,http://anzures.properati.com.mx/opct_venta_dep...,19.432657,-99.177444
3,7,sell,apartment,|Benito Juárez|Distrito Federal|México|,3150000.0,MXN,3122244.47,166001.78,NaN,100.0,NaN,31500.000000,NaN,NaN,NaN,http://benito-juarez.properati.com.mx/r3a9_ven...,19.367025,-99.170349
4,12,sell,apartment,|Benito Juárez|Distrito Federal|México|,1784569.0,MXN,1768425.20,94022.66,NaN,57.0,NaN,31308.228070,NaN,NaN,NaN,http://benito-juarez.properati.com.mx/pdji_ven...,23.634501,-102.552788
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3751,4613,sell,apartment,|Cuauhtémoc|Distrito Federal|México|,1150000.0,MXN,1139596.67,60589.45,NaN,65.0,NaN,17692.307690,NaN,NaN,NaN,http://cuauhtemoc.properati.com.mx/q59a_venta_...,23.634501,-102.552788
3752,4617,sell,apartment,|Azcapotzalco|Distrito Federal|México|,1785000.0,MXN,1768852.34,94045.37,NaN,65.0,NaN,27461.538460,NaN,NaN,NaN,http://azcapotzalco.properati.com.mx/o2tc_vent...,19.472155,-99.161186
3753,4621,sell,apartment,|Tlalpan|Distrito Federal|México|,1113000.0,MXN,1102931.38,58640.05,NaN,80.0,NaN,13912.500000,NaN,NaN,NaN,http://villa-lazaro-cardenas.properati.com.mx/...,19.295734,-99.146996
3754,4624,sell,apartment,|Cuauhtémoc|Distrito Federal|México|,739000.0,MXN,732314.77,38935.31,NaN,76.0,NaN,9723.684211,NaN,NaN,NaN,http://cuauhtemoc.properati.com.mx/izni_venta_...,23.634501,-102.552788


In [21]:
df.shape

(3756, 18)

In [22]:
df['currency'].value_counts()

currency
MXN    3756
Name: count, dtype: int64

In [23]:
df['property_type'].value_counts()

property_type
apartment    3756
Name: count, dtype: int64

Creating a scatter mapbox plot : visualise the distribution of houses based on cost

In [24]:
fig = px.scatter_mapbox(
    df,
    lat = 'lat',
    lon = 'lon',
    width = 900,
    height = 900,
    color = 'price_aprox_usd',
    hover_data = ['price_aprox_usd']
)

fig.update_layout(mapbox_style = "open-street-map")

to create a 3D scatter plot [install nbformat for this to work]

In [25]:
fig = px.scatter_3d(
    df,
    x = 'lon',
    y = 'lat',
    z = 'price_aprox_usd',
    labels = {'lon': "longitude", 'lat':'latitude','price_aprox_usd':'price'},
    width = 900,
    height= 900
)

fig.update_traces(
    marker = {'size':4, 'line': {'width': 2,'color': 'DarkSlateGrey'}},
    selector= {'mode':'markers'},
)

fig.show()

splitting data

In [26]:
features = ['lon','lat']
X_train = df[features]
X_train.shape

(3756, 2)

In [27]:
target = 'price_aprox_usd'
y_train = df[target]
y_train.shape

(3756,)

Build Model. to build a model, it is important to set a baseline to be able to evaluate modle's performance

In [28]:
y_mean = y_train.mean()
y_mean

102782.86024760384

In [29]:
y_pred_baseline = [y_mean] * len(y_train)
y_pred_baseline[:5]

[102782.86024760384,
 102782.86024760384,
 102782.86024760384,
 102782.86024760384,
 102782.86024760384]

In [30]:
mae_baseline = mean_absolute_error(y_train, y_pred_baseline)

print("mean apt price", round(y_mean,2))
print('Baseline MAE', round(mae_baseline,2))

mean apt price 102782.86
Baseline MAE 57345.98


iterate. a linear regression can't handle deviations where ther are missing values

In [31]:
imputer = SimpleImputer()

In [32]:
imputer.fit(X_train)

SimpleImputer()

In [34]:
XT_train = imputer.transform(X_train)
pd.DataFrame(XT_train, columns = XT_train.columns).info()

AttributeError: 'numpy.ndarray' object has no attribute 'columns'

In [35]:
model = make_pipeline(
    SimpleImputer(),
    LinearRegression()
)

In [36]:
model.fit(X_train, y_train)

Pipeline(steps=[('simpleimputer', SimpleImputer()),
                ('linearregression', LinearRegression())])

In [37]:
y_pred_training = model.predict(X_train)

In [38]:
mae_training = mean_absolute_error(y_train, y_pred_training)
round(mae_training,2)

56605.34